# First analysis osmFISH

In [62]:
from starfish import Experiment
from starfish import display
import numpy as np

In [48]:
path = r'Z:/Sabrina/Pre-fishcodes/osmFISH/Python_analysis_test/output/' 
experiment = Experiment.from_json(path + "experiment.json")

In [52]:
fov = experiment['fov_000']
image = fov.get_image("primary")
nuclei = fov.get_image("nuclei")

In [53]:
%gui qt
viewer = display(nuclei)
viewer.layers[0].name = "raw stack" # rename the layer

  0%|                                                    | 0/3 [00:00<?, ?it/s]C:\Users\tubau.n\.conda\envs\starfish\lib\site-packages\skimage\util\dtype.py:135: UserWarning: Possible precision loss when converting from float64 to float32
  .format(dtypeobj_in, dtypeobj_out))
100%|████████████████████████████████████████████| 3/3 [00:00<00:00,  4.30it/s]


# Registration

In [55]:
from starfish.image import ApplyTransform, LearnTransform
from starfish.types import Axes

def register(imgs, nuclei, method = 'translation'):
    mip_imgs = imgs.reduce(dims = [Axes.CH, Axes.ZPLANE], func="max")
    mip_dots = dots.reduce(dims = [Axes.CH, Axes.ZPLANE], func="max")
    learn_translation = LearnTransform.Translation(reference_stack=mip_dots, axes=Axes.ROUND, upsampling=1000)
    transforms_list = learn_translation.run(mip_imgs)
    warp = ApplyTransform.Warp()
    registered_imgs = warp.run(imgs, transforms_list=transforms_list, in_place=False, verbose=True)
    return registered_imgs

# Filtering

In [56]:
from starfish.image import Filter

def filter_white_tophat(imgs, nuclei, masking_radius):
    wth = Filter.WhiteTophat(masking_radius=masking_radius)
    return wth.run(imgs), wth.run(dots)

# Finding spots

In [57]:
from starfish.spots import FindSpots

def find_spots(imgs, nuclei):

    p = FindSpots.BlobDetector(
        min_sigma=1,
        max_sigma=10,
        num_sigma=30,
        threshold=0.01,
        measurement_type='mean',
    )

    intensities = p.run(image_stack=imgs, reference_image=dots)
    return intensities

# Decoding spot

In [58]:
from starfish.spots import DecodeSpots

def decode_spots(codebook, spots):
    decoder = DecodeSpots.PerRoundMaxChannel(codebook=codebook)
    return decoder.run(spots=spots)

# Segmenting cells

In [59]:
from starfish.image import Segment
from starfish.types import Axes

def segment(registered_imgs, nuclei):
    dapi_thresh = .22  # binary mask for cell (nuclear) locations
    stain_thresh = 0.011  # binary mask for overall cells // binarization of stain
    min_dist = 56

    registered_mp = registered_imgs.reduce(dims=[Axes.CH, Axes.ZPLANE], func="max").xarray.squeeze()
    stain = np.mean(registered_mp, axis=0)
    stain = stain / stain.max()
    nuclei = nuclei.reduce(dims=[Axes.ROUND, Axes.CH, Axes.ZPLANE], func="max")

    seg = Segment.Watershed(
        nuclei_threshold=dapi_thresh,
        input_threshold=stain_thresh,
        min_distance=min_dist
    )
    masks = seg.run(registered_imgs, nuclei)

    return seg, masks

# Single cell gene expression

In [60]:
from starfish.spots import AssignTargets

def make_expression_matrix(masks, decoded):
    al = AssignTargets.Label()
    labeled = al.run(masks, decoded[decoded.target != 'nan'])
    cg = labeled[labeled.cell_id != 'nan'].to_expression_matrix()
    return cg

# Pipeline

In [63]:
# filter
imgs_wth, dots_wth = filter_white_tophat(imgs, dots, 15)

# register
registered_imgs = register(imgs_wth, dots_wth)

# find spots
spots = find_spots(registered_imgs, dots_wth)

# decode
decoded = decode_spots(e.codebook, spots)

# segment
seg, masks = segment(registered_imgs, nuclei)

# make expression matrix
mat = make_expression_matrix(masks, decoded)

100%|████████████████████████████████████████████| 1/1 [00:00<00:00, 51.39it/s]


ValueError: not all masks have the same pixel ticks